## Imports

In [2]:
import sys
sys.path.append("../")
from autograd import numpy as np
from autograd import grad, jacobian
import pickle
import matplotlib.pyplot as plt
from SR_functions import Gamma, Dv, vadd, SinCosTheta, SinCosEpsilon, ABSC, E_eps, erf, Parameters, Lorentz, norm_squared
from twobox import TwoBox
from parameters import D1_ND
import pickle
_, L, m, c = Parameters()

I = 10e9
h = 1e-5
vx = (5/100)*c
v = np.array( [vx,0] )

In [3]:
grating_type = "Optimised"

if grating_type=="Ilic":
    ## Ilic
    wavelength      = 1.5 / D1_ND(5.3/100)
    grating_pitch   = 1.8 / wavelength
    grating_depth   = 0.5 / wavelength
    box1_width      = 0.15 * grating_pitch
    box2_width      = 0.35 * grating_pitch
    box_centre_dist = 0.60 * grating_pitch
    box1_eps        = 3.5**2 
    box2_eps        = 3.5**2
    gaussian_width_value  = 2 * 10
    substrate_depth = 0.5 / wavelength
    substrate_eps   = 1.45**2
if grating_type=="Optimised":
    ## Optimised - second
    grating_pitch   = 1.5384469388251338
    grating_depth   = 0.5580762361523982
    box1_width      = 0.10227122552871484
    box2_width      = 0.07605954942866577
    box_centre_dist = 0.2669020979549422
    box1_eps        = 9.614975107945112
    box2_eps        = 9.382304398409568
    gaussian_width_value  = 33.916288616522735
    substrate_depth = 0.17299998450776535
    substrate_eps   = 9.423032644325023

wavelength      = 1.
angle           = 0.
Nx              = 100
numG            = 25
Qabs            = np.inf

grating = TwoBox(grating_pitch, grating_depth, box1_width, box2_width, box_centre_dist, box1_eps, box2_eps, 
                 gaussian_width_value, substrate_depth, substrate_eps,
                 wavelength, angle, Nx, numG, Qabs)
w = gaussian_width_value

## Autograd forces

In [4]:
## Autograd
def fy(vx, vy, xM, yM, phiM):
    """
    ## Inputs
    t: Frame Mn time
    yvec: Frame Mn - [x, y, phi, vx, vy, vphi]
    vL: Frame L - [vx, vy]
    i: input step (for troubleshooting)
    ## Outputs
    Returns the vector d/dtau(yvec):\n
    [vx,vy,vphi,fx,fy,fphi]
    """
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,vy])

    ## Angles
    sintheta, costheta, theta = SinCosTheta(vL)

    delta    = theta - phiM

    sinphi   = np.sin(phiM)
    cosphi   = np.cos(phiM)

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength
    
    grating.wavelength = lam
    grating.angle = delta
    QR = grating.Q()
    Q1R = QR[0];     Q2R = QR[1]
    
    ## Restore
    grating.wavelength = lam
    grating.angle = input_angle

    grating.angle = - delta
    QL = grating.Q()
    Q1L = QL[0];     Q2L = -QL[1]
    

    ## Base factors
    A_int=yM*       ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + xM*       (g**2/(g+1))*(vx*vy)/c**2 
    B_int=cosphi*   ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + sinphi*   (g**2/(g+1))*(vx*vy)/c**2
    
    erfR=erf( (np.sqrt(2)/w)*( A_int + B_int*(L/2) ) )
    erfL=erf( (np.sqrt(2)/w)*( A_int - B_int*(L/2) ) )
    
    erfMID=erf( (np.sqrt(2)/w)*A_int )
    

    ## Integrals
    I0R=(w/(2*B_int))*np.sqrt(np.pi/2)* ( erfR - erfMID )
    I0L=-(w/(2*B_int))*np.sqrt(np.pi/2)*( erfL - erfMID )


    ## Forces
    fy=(1/m)*(D**2*I/c) * ( ( Q1R*sintheta + Q2R*costheta )*I0R + ( Q1L*sintheta + Q2L*costheta )*I0L )
    
    ## Inputs
    grating.wavelength = input_wavelength
    grating.angle = input_angle
    
    return fy

def fphi(vx, vy, xM, yM, phiM):
    """
    ## Inputs
    t: Frame Mn time
    yvec: Frame Mn - [x, y, phi, vx, vy, vphi]
    vL: Frame L - [vx, vy]
    i: input step (for troubleshooting)
    ## Outputs
    Returns the vector d/dtau(yvec):\n
    [vx,vy,vphi,fx,fy,fphi]
    """
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,vy])

    ## Angles
    sintheta, costheta, theta = SinCosTheta(vL)

    delta    = theta - phiM
    sindelta = np.sin(delta)
    cosdelta = np.cos(delta)

    sinphi   = np.sin(phiM)
    cosphi   = np.cos(phiM)

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength
    
    grating.wavelength = lam
    grating.angle = delta
    QR =grating.Q()
    Q1R = QR[0];     Q2R = QR[1]
    
    ## Restore
    grating.wavelength = lam
    grating.angle = input_angle

    grating.angle = - delta
    QL =grating.Q()
    Q1L = QL[0];     Q2L = -QL[1]
    

    ## Base factors
    A_int=yM*       ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + xM*       (g**2/(g+1))*(vx*vy)/c**2 
    B_int=cosphi*   ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + sinphi*   (g**2/(g+1))*(vx*vy)/c**2

    expR=np.exp(-2*(( A_int + B_int*(L/2) )**2)/w**2 )
    expL=np.exp(-2*(( A_int - B_int*(L/2) )**2)/w**2 )
    
    erfR=erf( (np.sqrt(2)/w)*( A_int + B_int*(L/2) ) )
    erfL=erf( (np.sqrt(2)/w)*( A_int - B_int*(L/2) ) )
    
    expMID=np.exp(-2*(A_int**2)/w**2 )
    erfMID=erf( (np.sqrt(2)/w)*A_int )
    
    
    I1R=(w/(4*B_int**2))* ( w*( expMID - expR ) - np.sqrt(2*np.pi)*A_int*( erfR - erfMID ) )
    I1L=(w/(4*B_int**2))* ( w*( expMID - expL ) - np.sqrt(2*np.pi)*A_int*( erfL - erfMID) )
    
    ## Forces
    fphi=-(12/(m*L**2))*(D**2*I/c)*( ( Q1R*cosdelta - Q2R*sindelta )*I1R - ( Q1L*cosdelta - Q2L*sindelta )*I1L )
    
    ## Inputs
    grating.wavelength = input_wavelength
    grating.angle = input_angle
    
    # return npa.array([fy,fphi])
    return fphi

## Algebraic derivative
def fy_phi(vx, vy, xM, yM, phiM):
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,vy])

    ## Angles
    sintheta, costheta, theta = SinCosTheta(vL)
    A, B, S, C = ABSC(vL,phiM)
    E = E_eps(vL,phiM)
    delta    = theta - phiM

    sinphi   = np.sin(phiM)
    cosphi   = np.cos(phiM)

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength

    ## Efficiency factors
    ## Right grating
    grating.wavelength  = lam 
    grating.angle       = delta
    Q1R, Q2R, dQ1ddeltaR, dQ2ddeltaR, dQ1dlambdaR, dQ2dlambdaR = grating.return_Qs_auto(True)
    
    ## Left grating
    grating.wavelength  = lam
    grating.angle       = -delta
    Q1R_neg, Q2R_neg, dQ1ddeltaR_neg, dQ2ddeltaR_neg, dQ1dlambdaR_neg, dQ2dlambdaR_neg = grating.return_Qs_auto(True)
    
    ## Symmetry
    Q1L = Q1R_neg;   Q2L = -Q2R_neg;   
    dQ1ddeltaL  = -dQ1ddeltaR_neg;    dQ2ddeltaL  = dQ2ddeltaR_neg
    dQ1dlambdaL = dQ1dlambdaR_neg;    dQ2dlambdaL = -dQ2dlambdaR_neg

    ## Define T_{pr,j}'
    T1R = (A/costheta - E) * dQ1ddeltaR + cosphi * lam * dQ1dlambdaR
    T1L = (A/costheta - E) * dQ1ddeltaL + cosphi * lam *dQ1dlambdaL
    T2R = (A/costheta - E) * dQ2ddeltaR + cosphi * lam *dQ2dlambdaR
    T2L = (A/costheta - E) * dQ2ddeltaL + cosphi * lam *dQ2dlambdaL

    ## Base factors
    A_int=yM*       ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + xM*       (g**2/(g+1))*(vx*vy)/c**2
    B_int=cosphi*   ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + sinphi*   (g**2/(g+1))*(vx*vy)/c**2

    expR=np.exp(-2*(( A_int + B_int*(L/2) )**2)/w**2 )
    expL=np.exp(-2*(( A_int - B_int*(L/2) )**2)/w**2 )
    
    erfR=erf( (np.sqrt(2)/w)*( A_int + B_int*(L/2) ) )
    erfL=erf( (np.sqrt(2)/w)*( A_int - B_int*(L/2) ) )
    
    expMID=np.exp(-2*(A_int**2)/w**2 )
    erfMID=erf( (np.sqrt(2)/w)*A_int )
    
    XR=A_int + B_int*(L/2)
    XL=A_int - B_int*(L/2)

    ## Integrals
    I0R=(w/(2*B_int))*np.sqrt(np.pi/2)* ( erfR - erfMID )
    I0L=-(w/(2*B_int))*np.sqrt(np.pi/2)*( erfL - erfMID )
    
    I1R=(w/(4*B_int**2))* ( w*( expMID - expR ) - np.sqrt(2*np.pi)*A_int*( erfR - erfMID ) )
    I1L=(w/(4*B_int**2))* ( w*( expMID - expL ) - np.sqrt(2*np.pi)*A_int*( erfL - erfMID) )
    
    I2R=(w/(16*B_int**3))* ( -4*w*(A_int*expMID - XL*expR) + np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfR - erfMID) )
    I2L=(w/(16*B_int**3))* (  4*w*(A_int*expMID - XR*expL) - np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfL - erfMID) )

    fy_phi_value = (1/m)*(D**2*I/c) *(1/c) * ( ( sintheta*( 2*cosphi*Q1R - T1R ) + costheta*( 2*cosphi*Q2R - T2R ) )*I1R
                                       - ( sintheta*( 2*cosphi*Q1L - T1L ) + costheta*( 2*cosphi*Q2L - T2L ) )*I1L
                                       + (-( A + costheta*E )*Q1R - ( B - sintheta*E )*Q2R)*I1R
                                       - (-( A + costheta*E )*Q1L - ( B - sintheta*E )*Q2L)*I1L
                           ) 
    grating.wavelength  = input_wavelength
    grating.angle       = input_angle

    return fy_phi_value

def fphi_phi(vx, vy, xM, yM, phiM):
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,vy])

    ## Angles
    sintheta, costheta, theta = SinCosTheta(vL)
    A, B, S, C = ABSC(vL,phiM)
    E = E_eps(vL,phiM)
    delta    = theta - phiM

    sinphi   = np.sin(phiM)
    cosphi   = np.cos(phiM)

    sindelta = np.sin(delta)
    cosdelta = np.cos(delta)

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength

    ## Efficiency factors
    ## Right grating
    grating.wavelength  = lam 
    grating.angle       = delta
    Q1R, Q2R, dQ1ddeltaR, dQ2ddeltaR, dQ1dlambdaR, dQ2dlambdaR = grating.return_Qs_auto(True)
    
    ## Left grating
    grating.wavelength  = lam
    grating.angle       = -delta
    Q1R_neg, Q2R_neg, dQ1ddeltaR_neg, dQ2ddeltaR_neg, dQ1dlambdaR_neg, dQ2dlambdaR_neg = grating.return_Qs_auto(True)
    
    ## Symmetry
    Q1L = Q1R_neg;   Q2L = -Q2R_neg;   
    dQ1ddeltaL  = -dQ1ddeltaR_neg;    dQ2ddeltaL  = dQ2ddeltaR_neg
    dQ1dlambdaL = dQ1dlambdaR_neg;    dQ2dlambdaL = -dQ2dlambdaR_neg

    ## Define T_{pr,j}'
    T1R = (A/costheta - E) * dQ1ddeltaR + cosphi * lam * dQ1dlambdaR
    T1L = (A/costheta - E) * dQ1ddeltaL + cosphi * lam *dQ1dlambdaL
    T2R = (A/costheta - E) * dQ2ddeltaR + cosphi * lam *dQ2dlambdaR
    T2L = (A/costheta - E) * dQ2ddeltaL + cosphi * lam *dQ2dlambdaL

    ## Base factors
    A_int = yM*       ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + xM*       (g**2/(g+1))*(vx*vy)/c**2
    B_int = cosphi*   ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + sinphi*   (g**2/(g+1))*(vx*vy)/c**2

    expR = np.exp(-2*(( A_int + B_int*(L/2) )**2)/w**2 )
    expL = np.exp(-2*(( A_int - B_int*(L/2) )**2)/w**2 )
    
    erfR = erf( (np.sqrt(2)/w)*( A_int + B_int*(L/2) ) )
    erfL = erf( (np.sqrt(2)/w)*( A_int - B_int*(L/2) ) )
    
    expMID = np.exp(-2*(A_int**2)/w**2 )
    erfMID = erf( (np.sqrt(2)/w)*A_int )
    
    XR = A_int + B_int*(L/2)
    XL = A_int - B_int*(L/2)

    ## Integrals
    I0R =  (w/(2*B_int))*np.sqrt(np.pi/2)* ( erfR - erfMID )
    I0L = -(w/(2*B_int))*np.sqrt(np.pi/2)* ( erfL - erfMID )
    
    I1R = (w/(4*B_int**2))* ( w*( expMID - expR ) - np.sqrt(2*np.pi)*A_int*( erfR - erfMID ) )
    I1L = (w/(4*B_int**2))* ( w*( expMID - expL ) - np.sqrt(2*np.pi)*A_int*( erfL - erfMID ) )
    
    I2R = (w/(16*B_int**3))* ( -4*w*(A_int*expMID - XL*expR) + np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfR - erfMID) )
    I2L = (w/(16*B_int**3))* (  4*w*(A_int*expMID - XR*expL) - np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfL - erfMID) )

    fphi_phi_value =-(12/(m*L**2))*(D**2*I/c)*(1/c)*( ( cosdelta*( 2*cosphi*Q1R - T1R )  )*I2R - sindelta*( 2*cosphi*Q2R - T2R )
                                                    + ( cosdelta*( 2*cosphi*Q1L - T1L )  )*I2L  - sindelta*( 2*cosphi*Q2L - T2L )
                                                    + (-( C - sindelta*E )*Q1R + ( S + cosdelta*E )*Q2R)*I1R
                                                    + (-( C - sindelta*E )*Q1L + ( S + cosdelta*E )*Q2L)*I1L
                                                                )

    grating.wavelength  = input_wavelength
    grating.angle       = input_angle

    return fphi_phi_value


fyy = grad(fy,argnum=3)
fyphi = grad(fy,argnum=4)
fphiy = grad(fphi,argnum=3)
fphiphi = grad(fphi,argnum=4)

muyy = grad(fy,argnum=1)
muphiy = grad(fphi,argnum=1)

## 
fyy_grad = fyy(vx, 0., 0., 0., 0.)
fyphi_grad = fyphi(vx, 0., 0., 0., 0.)
fphiy_grad = fphiy(vx, 0., 0., 0., 0.)
fphiphi_grad = fphiphi(vx, 0., 0., 0., 0.)

muyy_grad = muyy(vx, 0., 0., 0., 0.)
muyphi_grad = fy_phi(vx, 0., 0., 0., 0.)
muphiy_grad = muphiy(vx, 0., 0., 0., 0.)
muphiphi_grad = fphi_phi(vx, 0., 0., 0., 0.)

## Non-linear forces

In [5]:
def F(vx, vy, xM, yM, phiM, vphiM):
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,vy])

    ## Angles
    sintheta, costheta, theta = SinCosTheta(vL)
    A, B, S, C = ABSC(vL,phiM)
    E = E_eps(vL,phiM)
    delta    = theta - phiM

    sinphi   = np.sin(phiM)
    cosphi   = np.cos(phiM)

    sindelta = np.sin(delta)
    cosdelta = np.cos(delta)

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength

    ## Efficiency factors
    ## Right grating
    grating.wavelength  = lam 
    grating.angle       = delta
    Q1R, Q2R, dQ1ddeltaR, dQ2ddeltaR, dQ1dlambdaR, dQ2dlambdaR = grating.return_Qs_auto(True)
    
    ## Left grating
    grating.wavelength  = lam
    grating.angle       = -delta
    Q1R_neg, Q2R_neg, dQ1ddeltaR_neg, dQ2ddeltaR_neg, dQ1dlambdaR_neg, dQ2dlambdaR_neg = grating.return_Qs_auto(True)
    
    ## Symmetry
    Q1L = Q1R_neg;   Q2L = -Q2R_neg;   
    dQ1ddeltaL  = -dQ1ddeltaR_neg;    dQ2ddeltaL  = dQ2ddeltaR_neg
    dQ1dlambdaL = dQ1dlambdaR_neg;    dQ2dlambdaL = -dQ2dlambdaR_neg

    ## Define T_{pr,j}'
    T1R = (A/costheta - E) * dQ1ddeltaR + cosphi * lam * dQ1dlambdaR
    T1L = (A/costheta - E) * dQ1ddeltaL + cosphi * lam *dQ1dlambdaL
    T2R = (A/costheta - E) * dQ2ddeltaR + cosphi * lam *dQ2dlambdaR
    T2L = (A/costheta - E) * dQ2ddeltaL + cosphi * lam *dQ2dlambdaL

    ## Base factors
    A_int=yM*       ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + xM*       (g**2/(g+1))*(vx*vy)/c**2
    B_int=cosphi*   ( 1 + (g**2/(g+1))*(vy**2/c**2) )   + sinphi*   (g**2/(g+1))*(vx*vy)/c**2

    expR=np.exp(-2*(( A_int + B_int*(L/2) )**2)/w**2 )
    expL=np.exp(-2*(( A_int - B_int*(L/2) )**2)/w**2 )
    
    erfR=erf( (np.sqrt(2)/w)*( A_int + B_int*(L/2) ) )
    erfL=erf( (np.sqrt(2)/w)*( A_int - B_int*(L/2) ) )
    
    expMID=np.exp(-2*(A_int**2)/w**2 )
    erfMID=erf( (np.sqrt(2)/w)*A_int )
    
    XR=A_int + B_int*(L/2)
    XL=A_int - B_int*(L/2)

    ## Integrals
    I0R=(w/(2*B_int))*np.sqrt(np.pi/2)* ( erfR - erfMID )
    I0L=-(w/(2*B_int))*np.sqrt(np.pi/2)*( erfL - erfMID )
    
    I1R=(w/(4*B_int**2))* ( w*( expMID - expR ) - np.sqrt(2*np.pi)*A_int*( erfR - erfMID ) )
    I1L=(w/(4*B_int**2))* ( w*( expMID - expL ) - np.sqrt(2*np.pi)*A_int*( erfL - erfMID) )
    
    I2R = (w/(16*B_int**3))* ( -4*w*(A_int*expMID - XL*expR) + np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfR - erfMID) )
    I2L = (w/(16*B_int**3))* (  4*w*(A_int*expMID - XR*expL) - np.sqrt(2*np.pi)*(4*A_int**2 + w**2)* ( erfL - erfMID) )

    fy_value=(1/m)*(D**2*I/c) * ( ( Q1R*sintheta + Q2R*costheta )*I0R + ( Q1L*sintheta + Q2L*costheta )*I0L
                           + (vphiM/c)*( ( sintheta*( 2*cosphi*Q1R - T1R ) + costheta*( 2*cosphi*Q2R - T2R ) )*I1R
                                       - ( sintheta*( 2*cosphi*Q1L - T1L ) + costheta*( 2*cosphi*Q2L - T2L ) )*I1L
                                       + (-( A + costheta*E )*Q1R - ( B - sintheta*E )*Q2R)*I1R
                                       - (-( A + costheta*E )*Q1L - ( B - sintheta*E )*Q2L)*I1L
                           ) ) 
    
    fphi_value=-(12/(m*L**2))*(D**2*I/c)*( ( Q1R*cosdelta - Q2R*sindelta )*I1R - ( Q1L*cosdelta - Q2L*sindelta )*I1L 
                           + (vphiM/c)*( ( cosdelta*( 2*cosphi*Q1R - T1R ) - sindelta*( 2*cosphi*Q2R - T2R ) )*I2R 
                                       + ( cosdelta*( 2*cosphi*Q1L - T1L ) - sindelta*( 2*cosphi*Q2L - T2L ) )*I2L  
                                       + (-( C - sindelta*E )*Q1R + ( S + cosdelta*E )*Q2R)*I1R
                                       + (-( C - sindelta*E )*Q1L + ( S + cosdelta*E )*Q2L)*I1L
                           ) )
    grating.wavelength  = input_wavelength
    grating.angle       = input_angle

    return np.array( [fy_value,fphi_value] )

h_vy = 1

grad_y = ( F(vx,0.,0.,h,0.,0.) - F(vx,0.,0.,-h,0.,0.) ) /(2*h)
fyy_finite = grad_y[0];     fphiy_finite = grad_y[1]
grad_phi = ( F(vx,0.,0.,0.,h,0.) - F(vx,0.,0.,0.,-h,0.) ) /(2*h)
fyphi_finite = grad_phi[0]; fphiphi_finite = grad_phi[1]

grad_vy = ( F(vx,h_vy,0.,0.,0.,0.) - F(vx,-h_vy,0.,0.,0.,0.) ) /(2*h_vy)
muyy_finite = grad_vy[0];   muphiy_finite = grad_vy[1]
grad_vphi = ( F(vx,0.,0.,0.,0.,h) - F(vx,0.,0.,0.,0.,-h) ) /(2*h)
muyphi_finite = grad_vphi[0];      muphiphi_finite = grad_vphi[1]

## Algebraic

In [6]:
def algebraic(vx):
    ## Inputs
    input_wavelength = grating.wavelength
    input_angle = grating.angle

    vL = np.array([vx,0])

    ## Find  M wavelength
    D   = Dv(vL)
    g   = Gamma(vL)
    lam = wavelength / D  # incoming wavelength

    ## Efficiency factors
    ## Right grating
    grating.wavelength  = lam 
    grating.angle       = 0.
    Q1R, Q2R, dQ1ddeltaR, dQ2ddeltaR, dQ1dlambdaR, dQ2dlambdaR = grating.return_Qs_auto(True)
    
    ## Symmetry
    Q1L = Q1R;   Q2L = -Q2R;   
    dQ1ddeltaL  = -dQ1ddeltaR;    dQ2ddeltaL  = dQ2ddeltaR
    dQ1dlambdaL = dQ1dlambdaR;    dQ2dlambdaL = -dQ2dlambdaR

    w_bar = w / L

    # y acceleration
    fy_y= -     D**2 * (I/(m*c)) *  ( Q2R - Q2L ) * ( 1 - np.exp(-1/(2*w_bar**2) ) )
    fy_phi= -   D**2 * (I/(m*c)) * ( dQ2ddeltaR + dQ2ddeltaL ) * (w/2) * np.sqrt( np.pi/2 ) * erf( 1/(w_bar*np.sqrt(2)) )
    fy_vy= -    D**2 * (I/(m*c)) * (1/c) * ( (D+1)/(D*(g+1)) ) * ( Q1R + Q1L  + dQ2ddeltaR + dQ2ddeltaL ) * (w/2) * np.sqrt( np.pi/2 ) * erf( 1/(w_bar*np.sqrt(2)) )
    fy_vphi=    D**2 * (I/(m*c)) * (1/c) * ( 2*( Q2R - Q2L ) - lam*( dQ2dlambdaR - dQ2dlambdaL ) ) * (w/2)**2 * ( 1 - np.exp( -1/(2*w_bar**2) ))

    # phi acceleration
    fphi_y=     D**2 * (12*I/( m*c*L**2)) * ( Q1R + Q1L ) * (  (w/2)*np.sqrt( np.pi/2 )  * erf( 1/(w_bar*np.sqrt(2)))  - (L/2)* np.exp( -1/(2*w_bar**2) )  ) 
    fphi_phi=   D**2 * (12*I/( m*c*L**2)) * ( dQ1ddeltaR - dQ1ddeltaL - ( Q2R - Q2L ) ) * (w/2)**2 * ( 1 - np.exp( -1/(2*w_bar**2) ))
    fphi_vy=    D**2 * (12*I/( m*c*L**2)) * (1/c) * ( (D+1)/(D*(g+1)) ) * ( dQ1ddeltaR - dQ1ddeltaL - ( Q2R - Q2L ) ) * (w/2)**2 * ( 1 - np.exp( -1/(2*w_bar**2) ))
    fphi_vphi= -D**2 * (12*I/( m*c*L**2)) * (1/c) * ( 2*( Q1R + Q1L ) - lam*( dQ1dlambdaR + dQ1dlambdaL ) ) * (w/2)**2 * (  (w/2)*np.sqrt( np.pi/2 )  * erf( 1/(w_bar*np.sqrt(2)))  - (L/2)* np.exp( -1/(2*w_bar**2) )  ) 

    grating.wavelength  = input_wavelength
    grating.angle       = input_angle

    return (fy_y,fy_phi,fphi_y,fphi_phi) , (fy_vy, fy_vphi, fphi_vy, fphi_vphi)

rest, damp = algebraic(vx)

## Comparison

In [7]:
print("                 Autograd         ", "   Finite       ", "  Expressions  ")
print("k^y_y:     ", fyy_grad,      fyy_finite,         rest[0])
print("k^y_phi:   ", fyphi_grad,    fyphi_finite,       rest[1])
print("k^phi_y:   ", fphiy_grad,    fphiy_finite,       rest[2])
print("k^phi_phi: ",fphiphi_grad,   fphiphi_finite,     rest[3])
print()
print("mu^y_y:    ", muyy_grad,     muyy_finite,        damp[0])
print("mu^y_phi:  ", muyphi_grad,   muyphi_finite,      damp[1])
print("mu^phi_y:  ", muphiy_grad,   muphiy_finite,      damp[2])
print("mu^phi_phi:",muphiphi_grad,  muphiphi_finite,    damp[3])

## difference between mu^y_y is just 1/gammma (due to frame U)

                 Autograd             Finite          Expressions  
k^y_y:      -719.9290250826461 -719.9290261763955 -719.9290250826482
k^y_phi:    -88704.51783949493 -88704.51794922557 -88704.51783949505
k^phi_y:    323.45759139457914 323.45759138887445 323.45759139457715
k^phi_phi:  -51085.76197873926 -51085.761446926066 -51085.76197873924

mu^y_y:     -0.0006258264347129157 -0.0006258268610457256 -0.0006250436621305079
mu^y_phi:   -0.004042217233416669 -0.0040422172334166695 -0.004042217233416668
mu^phi_y:   -0.00017488526298349805 -0.00017488548272792532 -0.0001746665196046032
mu^phi_phi: -0.003203032171962367 -0.0032030321719623673 -0.003203032171962373


In [8]:
J = np.array([  [0,0,1,0],
                [0,0,0,1],
                [rest[0], rest[1], damp[0], damp[1]],
                [rest[2], rest[3], damp[2], damp[3]] ])
eigs = np.linalg.eig(J)[0]
print(np.real(eigs), np.imag(eigs))

[-0.00176074 -0.00176074 -0.0001533  -0.0001533 ] [ 224.74317262 -224.74317262   36.00274106  -36.00274106]


## Previous

In [9]:
rest_old, damp_old = grating.Linear_info_new(1/D1_ND(v/c),I)[1:3]
print(rest_old[0])
print(rest_old[1])
print(rest_old[2])
print(rest_old[3])
print()
print(damp_old[0])
print(damp_old[1])
print(damp_old[2])
print(damp_old[3])


J = np.array([  [0,0,1,0],
                [0,0,0,1],
                [rest_old[0], rest_old[1], damp_old[0], damp_old[1]],
                [rest_old[2], rest_old[3], damp_old[2], damp_old[3]] ])
eigs = np.linalg.eig(J)[0]
print(np.real(eigs), np.imag(eigs))

-719.9290250826482
-88704.51783949505
323.45759139457715
-51085.76197873924

-0.0006250436621305079
-0.004042217233416668
-0.0001746665196046032
-0.003203032171962373
[-0.00176074 -0.00176074 -0.0001533  -0.0001533 ] [ 224.74317262 -224.74317262   36.00274106  -36.00274106]


In [10]:
grating.wavelength = 1/D1_ND(v/c)
real, imag = grating.Eigs(I,m,c,"grad",False,False)
grating.wavelength = 1.
print(real, imag)

[-0.00176074 -0.00176074 -0.0001533  -0.0001533 ] [ 224.74317262 -224.74317262   36.00274106  -36.00274106]
